In [1]:
from datetime import datetime
from aeon.classification.convolution_based import RocketClassifier
from aeon.classification.feature_based import Catch22Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from aeon.classification.deep_learning import InceptionTimeClassifier
from aeon.classification.deep_learning import ResNetClassifier

from f1_etl import (
    DataConfig,
    SessionConfig,
    create_safety_car_dataset,
)
from f1_etl.train import (
    ModelEvaluationSuite,
    create_metadata_from_f1_dataset,
    prepare_data_with_validation,
    create_model_metadata,
    train_and_validate_model,
    evaluate_on_external_dataset,
    compare_performance_across_datasets,
)

In [2]:
drivers=["1", "2"]

# 1. Load dataset
data_config = DataConfig(
    sessions=[
        SessionConfig(2024, "Qatar Grand Prix", "R"),
    ],
    drivers=drivers,
    include_weather=False,
)

dataset = create_safety_car_dataset(
    config=data_config,
    window_size=50,
    prediction_horizon=100,
    normalize=True,
    target_column="TrackStatus",
    # resampling_strategy="adasyn",
)


2025-07-11 20:46:57,992 - f1_etl - INFO - Preprocessing configuration:
2025-07-11 20:46:58,011 - f1_etl - INFO -   Missing values: enabled (forward_fill)
2025-07-11 20:46:58,023 - f1_etl - INFO -   Normalization: enabled (standard)
2025-07-11 20:46:58,026 - f1_etl - INFO -   Resampling: disabled
2025-07-11 20:46:58,037 - f1_etl - INFO - Driver configuration:
2025-07-11 20:46:58,044 - f1_etl - INFO -   Global drivers: ['1', '2']
2025-07-11 20:46:58,049 - f1_etl - INFO -   Qatar Grand Prix: ['1', '2']
req         WARNING 	DEFAULT CACHE ENABLED! (4.94 GB) /Users/seansica/Library/Caches/fastf1


Loading session: 2024 Qatar Grand Prix R


events      WARNING 	Correcting user input 'Qatar Grand Prix' to 'Qatar Grand Prix'
core           INFO 	Loading data for Qatar Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '81', '63', '10', '55', '14', '24', '20', '4', '77', '44', '22', '30', '23', '27', '1


📊 Track Status Analysis (training_data):
   green       : 57940 samples ( 83.7%)
   safety_car  : 10503 samples ( 15.2%)
   vsc         :   146 samples (  0.2%)
   yellow      :   613 samples (  0.9%)
   Missing classes: [np.str_('red'), np.str_('unknown'), np.str_('vsc_ending')]
✅ FixedVocabTrackStatusEncoder fitted
   Classes seen: ['green', 'safety_car', 'vsc', 'yellow']
   Total classes: 7
   Output mode: integer labels


2025-07-11 20:52:00,787 - f1_etl - INFO - Total sequences generated: 2763
2025-07-11 20:52:00,792 - f1_etl - INFO - Generated 2763 sequences with shape (2763, 50, 9)
2025-07-11 20:52:00,793 - f1_etl - INFO - No missing values detected, skipping imputation
2025-07-11 20:52:00,793 - f1_etl - INFO - Applying normalization with method: standard
2025-07-11 20:52:00,807 - f1_etl - INFO - Final dataset summary:
2025-07-11 20:52:00,808 - f1_etl - INFO -   Sequences: 2763
2025-07-11 20:52:00,809 - f1_etl - INFO -   Features: 9
2025-07-11 20:52:00,809 - f1_etl - INFO -   Classes: 7 (integer)
2025-07-11 20:52:00,810 - f1_etl - INFO -   Label shape: (2763,)
2025-07-11 20:52:00,810 - f1_etl - INFO -     green       :  2312 samples ( 83.7%)
2025-07-11 20:52:00,810 - f1_etl - INFO -     safety_car  :   420 samples ( 15.2%)
2025-07-11 20:52:00,811 - f1_etl - INFO -     vsc         :     6 samples (  0.2%)
2025-07-11 20:52:00,811 - f1_etl - INFO -     yellow      :    25 samples (  0.9%)
